# Entrenamiento de modelo para deteccion de matriculas.

Ese notebook esta basado en el script de ejemplo del turorial de TensorFlow ["Retrain EfficientDet for the Edge TPU with TensorFlow Lite Model Maker"](https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_efficientdet_model_maker_tf2.ipynb#scrollTo=license), modificado para que funcione en el entorno de Google Colab. Las modificaciones se basan en en la solucion aportada en el siguiente [Issue](https://github.com/tensorflow/tensorflow/issues/60431#issuecomment-2239891537) de la libreria de TensorFlow Lite Model Maker.

## Preparación del entorno

### 1. Instalación de Conda, con Python 3.9
A dia de 24/09/2024, la version más nueva de Python compatible con la libreria de TFLite Model Maker. Ya que el entorno de de Google Colab solo trabaja con versiones posteriores, es necesario crear un entorno virtual. Para ello, se utiliza Conda, un sistema de gestión de paquetes popular para el lenguaje Python.

In [ ]:
# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local

# Update Conda
!conda update -n base -c defaults conda -y

# Create a Python 3.9 environment
!conda create --name py39_environment python=3.9 -y

# Initialize shell for Conda
!conda init bash

# Activate the environment and check Python version
!source activate py39_environment && python --version

--2024-10-12 08:24:33--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90040905 (86M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’

Miniconda3-py37_4.9 100%[===================>]  85.87M   102MB/s    in 0.8s    

2024-10-12 08:24:34 (102 MB/s) - ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’ saved [90040905/90040905]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py37h261ae71_2
    - chardet==3.0.4=py37h06a4308_1003
    - conda-

Tras instalar Conda y crear el entorno virtual, se isntalan los paquetes necesarios. También hace falta instalar la version 11 de CUDA compatible con la version de Tensorflow y TFLite Model Maker utilizadas en este script.

In [ ]:
!source activate py39_environment && python --version
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install tflite_model_maker
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install -q pycocotools
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install opencv-python-headless
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install ipykernel
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install --upgrade numba llvmlite
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && pip install tensorflow_io==0.23.1

# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
# Descargar el archivo de preferencia de CUDA:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin

#Mueve el archivo descargado a la ubicación correcta para que el sistema reconozca las preferencias de CUDA durante la instalación.
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600

#Descargar el instalador de CUDA del repositorio
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

#Instalar el paquete descargado
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

#Agregar la clave del repositorio
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub

#Actualizar la lista de paquetes e instalar el toolkit de CUDA
!apt-get update && sudo apt-get install cuda-toolkit-11-0

#Establecer la variable de entorno, para que TF encuentre la biblioteca
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH


Python 3.9.20
  Preparing metadata (setup.py) ... done
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached kaggle-1.6.17.tar.gz (82 kB)
  Preparing metadata (setup.py) ... done
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.

## 2. Obtención y preparación del dataset.
El [dataset](https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e/dataset/4) utilizado se puede encontrar en la plataforma de Roboflow Universe. Este conjuto es un extenso conjunto de imagenes de distintos tipos vehiculos de vehiculos, en distintas situaciones. donde se pueden observar sus matriculas. Se ha elegido debido a su gran tamaño, además de buena calidad, al tener mucha variedad en cuanto a la posición de la matricula. Todo esto aporta robustez al modelo.

Por motivos de comodidad, se ha subido el dataset a una carpeta publica de Google Drive, para que se pueda descargar sin necesidad de algún tipo de autenticación.

In [ ]:
#Borrado de la carpeta: Para prevenir errores si se quiere volver a ejecutar la celda si reiniciar el entorno de ejecución.
!sudo rm -r dataset

# Descarga del archivo zip del dataset subido en Google Drive
!gdown 1rHMNaFuG8yoNES-WGQPzaaptl-jY_UeQ ; unzip 'License Plate Detection MERGED PROJECTS.v3-best-model.voc.zip'; rm 'License Plate Detection MERGED PROJECTS.v3-best-model.voc.zip'

Streaming output truncated to the last 5000 lines.
 extracting: train/ibwwlr57edma1_jpg.rf.703c464878ccf230aebbed6a3712353d.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.718b4e5a968bdfb9c99a6b0ce6bd11a1.jpg  
 extracting: train/ibwwlr57edma1_jpg.rf.718b4e5a968bdfb9c99a6b0ce6bd11a1.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.7edc828972af7824fedeeb13c8f6071d.jpg  
 extracting: train/ibwwlr57edma1_jpg.rf.7edc828972af7824fedeeb13c8f6071d.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.80cd356e6214f9a87ce117b0a9ac6dc7.jpg  
 extracting: train/ibwwlr57edma1_jpg.rf.80cd356e6214f9a87ce117b0a9ac6dc7.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.85aedeb409581049220363f9020a8e4e.jpg  
 extracting: train/ibwwlr57edma1_jpg.rf.85aedeb409581049220363f9020a8e4e.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.88bfeefd699dc0f9e08a7e9c547909fa.jpg  
 extracting: train/ibwwlr57edma1_jpg.rf.88bfeefd699dc0f9e08a7e9c547909fa.xml  
 extracting: train/ibwwlr57edma1_jpg.rf.925dd7a3e9d73d54105984faf9a60783.jpg  
 

La notación Pascal VOC (Visual Object Classes Challenge) es un estándar utilizado principalmente en el campo de la detección de objetos y la segmentación de imágenes. Fue introducida como parte de un desafío organizado por la comunidad de visión por computadora para evaluar modelos y algoritmos en tareas de reconocimiento y localización de objetos.

Las anotaciones se suelen almacenar en archivos XML para cada imagen, donde se describen los objetos presentes en la imagen y su posición. Cada archivo XML contiene información como:

   - **Tamaño de la imagen**: Dimensiones de la imagen (ancho y alto).
   - **Nombre de la imagen**: Nombre del archivo de la imagen.
   - **Objetos detectados**: Para cada objeto, se incluye información como:
     *  Nombre de la clase: La categoría del objeto (por ejemplo, "perro", "gato", "persona")
     * Bounding box: Coordenadas del rectángulo delimitador que rodea al objeto, generalmente en el formato (xmin, ymin, xmax, ymax), donde:
       -  *xmin* y *ymin* son las coordenadas de la esquina superior izquierda.
       -  *xmax* y *ymax* son las coordenadas de la esquina inferior derecha.
               

Aunque las imagenes del dataset ya estan divididos en conjuntos de entrenamiento, validación y prueba, es necesario también dividir las imagenes de las anotaciones. En el siguiente script bash se realiza esta división.

In [ ]:
%%bash

mkdir -p dataset/train/images
mkdir -p dataset/train/annotations

mkdir -p dataset/validation/images
mkdir -p dataset/validation/annotations

mkdir -p dataset/test/images
mkdir -p dataset/test/annotations

mv train/*.xml dataset/train/annotations
mv train/*.jpg dataset/train/images

mv valid/*.xml dataset/validation/annotations
mv valid/*.jpg dataset/validation/images

mv test/*.xml dataset/test/annotations
mv test/*.jpg dataset/test/images

rm -r test
rm -r train
rm -r valid

rm READ*

## 3. Script de entrenamiento

### Establecimiento del nombre del modelo.

En la siguiente celda se establece el nombre que se quiere dar al archivo del modelo. Se realiza mediante una variable de entorno, ya que se utiliza tanto en el script de entrenamiento, como en la compilacion del modelo para el Edge TPU.

In [ ]:
TFLITE_FILENAME = 'license-detector'
%env TFLITE_FILENAME=$TFLITE_FILENAME

env: TFLITE_FILENAME=license-detector


### Script de entrenamiento.

La siguiente celda contiene el codigo en Python de entrenamiento. Es necesario tenerlo todo en un archivo, para poder ejecutarlo en el entorno creado.

In [ ]:
%%writefile my_script.py
import numpy as np
import os
import os
import random
import shutil

np.object = object
np.bool = bool
np.complex = complex

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector


import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

#Listar los modelos de detección de objetos
object_detection_models = model_spec.OBJECT_DETECTION_MODELS
print(object_detection_models)


# Your labels map as a dictionary (zero is reserved):
label_map = {1: 'license'}
print(label_map)

# If your dataset is already split, specify each path:
train_images_dir = 'dataset/train/images'
train_annotations_dir = 'dataset/train/annotations'
val_images_dir = 'dataset/validation/images'
val_annotations_dir = 'dataset/validation/annotations'
test_images_dir = 'dataset/test/images'
test_annotations_dir = 'dataset/test/annotations'


# We need to instantiate a separate DataLoader for each split dataset
train_data = object_detector.DataLoader.from_pascal_voc(
    train_images_dir, train_annotations_dir, label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(
    val_images_dir, val_annotations_dir, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(
    test_images_dir, test_annotations_dir, label_map=label_map)

# Obtener y mostrar el tamaño de cada DataLoader
print(f"Tamaño del conjunto de entrenamiento: {train_data.size}")
print(f"Tamaño del conjunto de validación: {validation_data.size}")
print(f"Tamaño del conjunto de prueba: {test_data.size}")

spec = model_spec.get('efficientdet_lite1')
print('TFLITE_FILENAME: ', os.getenv("TFLITE_FILENAME"))

model = object_detector.create(train_data=train_data,
                               model_spec=spec,
                               validation_data=validation_data,
                               epochs=10,
                               batch_size=25,
                               train_whole_model=True)

metrics = model.evaluate(test_data)

# Mostrar las métricas obtenidas
print("Métricas de evaluación del modelo cargado:")
for key, value in metrics.items():
    print(f"{key}: {value}")


TFLITE_FILENAME = f'{os.getenv("TFLITE_FILENAME")}.tflite'
LABELS_FILENAME = 'labels.txt'

model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             saved_model_filename='saved_model', export_format=[ExportFormat.TFLITE, ExportFormat.LABEL, ExportFormat.SAVED_MODEL] )

metrics = model.evaluate_tflite(TFLITE_FILENAME, test_data)

# Mostrar las métricas obtenidas
print("Métricas de evaluación del modelo cargado:")
for key, value in metrics.items():
    print(f"{key}: {value}")


Writing my_script.py


### Ejecución del script en el entorno creado
Tras crear el script, se ejecutan los siguientes comando para activar el entorno y ejecutar el programa de entrenamiento.

In [ ]:
!source /usr/local/etc/profile.d/conda.sh && conda activate py39_environment && python my_script.py

/usr/local/envs/py39_environment/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/envs/py39_environment/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an

Tras obtener el modelo entrenado, se descarga el compilador para el Edge TPU.

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0   7199      0 --:--:-- --:--:-- --:--:--  7248
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 file:/var/cuda-repo-ubuntu1804-11-0-local  InRelease
Ign:1 file:/var/cuda-repo-ubuntu1804-11-0-local  InRelease
Get:2 file:/var/cuda-repo-ubuntu1804-11-0-local  Release [564 B]
Get:2 file:/var/cuda-repo-ubuntu1804-11-0-local  Release [564 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [1,423 B]
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:

In [ ]:
import os
NUMBER_OF_TPUS =  1
LABELS_FILENAME = 'labels.txt'
print('>> ', os.getenv('TFLITE_FILENAME'))
TFLITE_FILENAME = os.getenv('TFLITE_FILENAME')
!edgetpu_compiler "{TFLITE_FILENAME}.tflite" -d --num_segments=$NUMBER_OF_TPUS


>>  license-detector
Edge TPU Compiler version 16.0.384591198
Searching for valid delegate with step 1
Try to compile segment with 322 ops
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5014 ms.

Input model: license-detector.tflite
Input size: 5.66MiB
Output model: license-detector_edgetpu.tflite
Output size: 7.57MiB
On-chip memory used for caching model parameters: 5.92MiB
On-chip memory remaining for caching model parameters: 1.58MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 322
Operation log: license-detector_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 319
Numbe

In [ ]:
# Use the Python variable in the shell command by prefixing with $
!find . -name "*.tflite" | zip "{TFLITE_FILENAME}_edge.zip" -@
!zip -r "{TFLITE_FILENAME}_savedModel.zip" saved_model/


  adding: license-detector_edgetpu.tflite (deflated 41%)
  adding: license-detector.tflite (deflated 28%)
  adding: saved_model/ (stored 0%)
  adding: saved_model/assets/ (stored 0%)
  adding: saved_model/saved_model.pb (deflated 91%)
  adding: saved_model/variables/ (stored 0%)
  adding: saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: saved_model/variables/variables.index (deflated 84%)


In [ ]:
import os
from google.colab import files
# Get a list of all files in the current directory
files_list = os.listdir()

# Filter the list of files to only include ZIP files
zip_files = [file for file in files_list if file.endswith(".zip")]

# Download all of the ZIP files
for zip_file in zip_files:
  files.download(zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>